In [1]:
import BandaiDataset as bd

In [2]:
MODEL_DIR = "./saved_models"
DATASET_DIR = "./datasets/data/"
FILELIST_PATH = "datafiles.txt"

In [3]:
dataset = bd.BandaiDataset(FILELIST_PATH)

In [4]:
import torch
set_frame = 50
def custom_collate_fn(batch):
    motion_batch_tensor = torch.FloatTensor(len(batch),50,480,640)
    motion_tensors = []
    labels = []
    #print(type(batch))

    for item in batch:
        #print(item)
        motion_tensor = item.get_motion_tensor(50) # load an motion as a tensor(frames,width,height)
        motion_tensors.append(motion_tensor.unsqueeze(0)) # put motions into a list : to be checked 
        labels.append(item.label)

    torch.cat(motion_tensors, out=motion_batch_tensor)
    label_batch_tensor = torch.LongTensor(labels)
    return (motion_batch_tensor,label_batch_tensor)

In [5]:
from torch.utils.data import DataLoader, random_split
import multiprocessing as mp

def load_data(file_list_path= '', data_path='', batch_sz = 5, train_val_test_split = [0.7,0.1,0.2]):
    assert sum(train_val_test_split) == 1, "Train, val and test fractions should sum to 1!" 
    dataset = bd.BandaiDataset(data_path)
    dataset.load()

    tr_va_te = []
    n_cpus = mp.cpu_count()
    
    for frac in train_val_test_split:
        num = round(frac * dataset.num_of_files)
        tr_va_te.append(num)
    
    if tr_va_te[0] != (dataset.num_of_files - tr_va_te[1] - tr_va_te[2]):
        tr_va_te[0] = (dataset.num_of_files - tr_va_te[1] - tr_va_te[2])
    
    #assert 1==2, f"tr_va_te = {tr_va_te}"

    train_split, val_split, test_split = random_split(dataset, tr_va_te)

    train_dl = DataLoader(train_split,
                          batch_size=batch_sz,
                          shuffle=True,
                          collate_fn=custom_collate_fn,
                          num_workers=n_cpus
                        )
    val_dl = DataLoader(val_split,
                        batch_size=batch_sz,
                        shuffle=True,
                        collate_fn=custom_collate_fn,
                        num_workers=n_cpus)
    test_dl = DataLoader(test_split,
                         batch_size=batch_sz,
                         shuffle=True,
                         collate_fn=custom_collate_fn,
                         num_workers=n_cpus)

    return train_dl, val_dl, test_dl

In [6]:
train_dl, val_dl, test_dl = load_data()

In [7]:
train_motions, _ = next(iter(train_dl))

In [8]:
test_motions,test_labels = next(iter(test_dl))

In [9]:
'''
for (motion_batch,label_batch) in train_dl:
    batch_sz = len(motion_batch)
    print(batch_sz)
'''

'\nfor (motion_batch,label_batch) in train_dl:\n    batch_sz = len(motion_batch)\n    print(batch_sz)\n'

In [10]:
import numpy as np
print(np.size(dataset[164].pose_list[0]))
print(dataset[164].frame_num)
#tensor = np.array(dataset[54].pose_list)
for pose in dataset[54].pose_list:
    print(np.size(pose))

307200
227
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200
307200


In [11]:
print(len(dataset.filelist))
print(dataset.filelist[164])
dataset.num_of_files


165
dataset-1_walk_tired_002


165

In [12]:
for i in range(0,165):
    print(f'{i}: {dataset[i].frame_num} ->' ,end='')
    tensor = dataset[i].get_motion_tensor(50)
    print({dataset[i].frame_num},flush=True)

0: 129 ->{50}
1: 91 ->{50}
2: 99 ->{50}
3: 118 ->{50}
4: 219 ->{50}
5: 123 ->{50}
6: 139 ->{50}
7: 71 ->{50}
8: 157 ->{50}
9: 139 ->{50}
10: 146 ->{50}
11: 173 ->{50}
12: 138 ->{50}
13: 222 ->{50}
14: 166 ->{50}
15: 230 ->{50}
16: 56 ->{50}
17: 170 ->{50}
18: 150 ->{50}
19: 85 ->{50}
20: 236 ->{50}
21: 221 ->{50}
22: 202 ->{50}
23: 158 ->{50}
24: 122 ->{50}
25: 141 ->{50}
26: 123 ->{50}
27: 80 ->{50}
28: 235 ->{50}
29: 304 ->{50}
30: 186 ->{50}
31: 111 ->{50}
32: 124 ->{50}
33: 131 ->{50}
34: 329 ->{50}
35: 143 ->{50}
36: 224 ->{50}
37: 213 ->{50}
38: 188 ->{50}
39: 112 ->{50}
40: 209 ->{50}
41: 142 ->{50}
42: 174 ->{50}
43: 254 ->{50}
44: 211 ->{50}
45: 86 ->{50}
46: 75 ->{50}
47: 76 ->{50}
48: 70 ->{50}
49: 98 ->{50}
50: 68 ->{50}
51: 118 ->{50}
52: 71 ->{50}
53: 68 ->{50}
54: 21 ->{50}
55: 100 ->{50}
56: 86 ->{50}
57: 106 ->{50}
58: 82 ->{50}
59: 94 ->{50}
60: 141 ->{50}
61: 111 ->{50}
62: 120 ->{50}
63: 187 ->{50}
64: 301 ->{50}
65: 164 ->{50}
66: 106 ->{50}
67: 153 ->{50}
68: 201 

In [13]:
import torchmetrics.classification as tmcls 
class ClassifierMetrics(object):
    ap: float
    precision: float
    recall: float
    f1: float
    acc: float
    count: int

    def __init__(self, task, n_labels, device):
        self.task = task
        if self.task == "multiclass":
            self.ap_metric = tmcls.MulticlassAveragePrecision(num_classes=n_labels, average=None, thresholds=None).to(device)
            self.precision_metric = tmcls.MulticlassPrecision(num_classes=n_labels).to(device)
            self.recall_metric = tmcls.MulticlassRecall(num_classes=n_labels).to(device)
            self.f1_metric = tmcls.MulticlassF1Score(num_classes=n_labels).to(device)
            self.acc_metric = tmcls.MulticlassAccuracy(num_classes=n_labels).to(device)

        elif self.task == "multilabel":
            self.ap_metric = tmcls.MultilabelAveragePrecision(num_labels=n_labels, average=None, thresholds=None).to(device)
            self.precision_metric = tmcls.MultilabelPrecision(num_labels=n_labels).to(device)
            self.recall_metric = tmcls.MultilabelRecall(num_labels=n_labels).to(device)
            self.f1_metric = tmcls.MultilabelF1Score(num_labels=n_labels).to(device)
            self.acc_metric = tmcls.MultilabelAccuracy(task=self.task, num_labels=n_labels).to(device)
        self.reset()
    

    def reset(self):
        self.ap = 0
        self.precision = 0
        self.recall = 0
        self.f1 = 0
        self.acc = 0
        self.count = 0

    def update(self, y_pred, y):
        y = y.long()
        self.ap += self.ap_metric(y_pred, y)
        self.precision += self.precision_metric(y_pred, y)
        self.recall += self.recall_metric(y_pred, y)
        self.f1 += self.f1_metric(y_pred, y)
        self.acc += self.acc_metric(y_pred, y)
        self.count += 1 #y.size(0)

    def calc(self, y_pred, y):
        self.reset()
        y = y.long()
        self.ap = self.ap_metric(y_pred, y)
        self.precision = self.precision_metric(y_pred, y)
        self.recall = self.recall_metric(y_pred, y)
        self.f1 = self.f1_metric(y_pred, y)

    def avg(self):
        self.ap = self.ap / self.count
        self.precision = self.precision / self.count
        self.recall = self.recall / self.count
        self.f1 = self.f1 / self.count
        self.acc = self.acc / self.count

In [23]:
import torch.nn as nn
from torchsummary import summary

class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=5, stride=1, padding=2) #[(dimension_sz−kernel+2*Padding)/Stride]+1
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=5, stride=1, padding=2) #[(dimension_sz−kernel+2*Padding)/Stride]+1
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)  
##########
        '''
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=5, stride=1, padding=2) #[(dimension_sz−kernel+2*Padding)/Stride]+1
        self.relu3 = nn.ReLU()
        '''
        #self.pool3 = nn.MaxPool2d(kernel_size=2)  
        
        self.flat = nn.Flatten()            
        self.fc1 = nn.Linear(50 * 160 * 120, 128)   
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(128, 256)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(256, 64)
        self.relu6 = nn.ReLU()

        self.fc4 = nn.Linear(64, 10)

    def forward(self, inp):
        inp = self.conv1(inp)
        inp = self.relu1(inp) 
        inp = self.pool1(inp)

        inp = self.conv2(inp)
        inp = self.relu2(inp) 
        inp = self.pool2(inp)
        '''
        inp = self.conv3(inp)
        inp = self.relu3(inp) 
        '''
        #inp = self.pool3(inp)
        

        inp = self.flat(inp)

        inp = self.fc1 (inp) 
        inp = self.relu4(inp)            
        inp = self.fc2(inp) 
        inp = self.relu5(inp)
        inp = self.fc3(inp) 
        inp = self.relu6(inp)
        out = self.fc4(inp)
        return out
    

def get_simple_conv_net():
    return ConvNet()

summary(get_simple_conv_net(), input_size=(50, 640, 480), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 50, 640, 480]          62,550
              ReLU-2         [-1, 50, 640, 480]               0
         MaxPool2d-3         [-1, 50, 320, 240]               0
            Conv2d-4         [-1, 50, 320, 240]          62,550
              ReLU-5         [-1, 50, 320, 240]               0
         MaxPool2d-6         [-1, 50, 160, 120]               0
           Flatten-7               [-1, 960000]               0
            Linear-8                  [-1, 128]     122,880,128
              ReLU-9                  [-1, 128]               0
           Linear-10                  [-1, 256]          33,024
             ReLU-11                  [-1, 256]               0
           Linear-12                   [-1, 64]          16,448
             ReLU-13                   [-1, 64]               0
           Linear-14                   

In [24]:
def train_model(model,epochs,train_dl,optimiser, device):
    msg = ""
    for epoch in range (epochs):
        total_steps = len(train_dl)
        correct = 0
        total = 0
        model.train()
        for batch_num, (motion_batch, label_batch) in enumerate(train_dl):
            batch_sz = len(motion_batch)
            motion_batch  = motion_batch.to(device)
            label_batch = label_batch.to(device)
            output = model(motion_batch)
            losses = nn.CrossEntropyLoss()(output, label_batch)
            optimiser.zero_grad()
            losses.backward
            optimiser.step()

            preds = torch.argmax(output,dim=1)
            correct += int(torch.eq(preds,label_batch).sum())
            total += batch_sz
            minibatch_accuracy = 100 * correct/total

            #### Fancy printing stuff, you can ignore this! ######
            if (batch_num + 1) % 5 == 0:
                print(" " * len(msg), end='\r')
                msg = f'Train epoch[{epoch+1}/{epochs}], MiniBatch[{batch_num + 1}/{total_steps}], Loss: {losses.item():.5f}, Acc: {minibatch_accuracy:.5f}'
                print (msg, end='\r' if epoch < epochs else "\n",flush=True)
            #### Fancy printing stuff, you can ignore this! ######
                
            

In [25]:
from torch.optim import SGD

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
epochs = 50
batch_sz = 16
learning_rate = 0.00005
train_dl, val_dl, test_dl = load_data(batch_sz=batch_sz)
network = get_simple_conv_net().to(DEVICE)
optim = SGD(network.parameters(),lr=learning_rate)



cuda


In [26]:
train_model(network,epochs,train_dl,optim, DEVICE)